In [1]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [36]:
pd.set_option('display.max_rows', 65)

In [2]:
data_path = "/Users/rcalef/sandbox/courses/data_vis_6.C85/a2/condo_conversions_data"
boston_path = os.path.join(data_path, "Boston", "boston_condo_conversions.csv")
cambridge_path = os.path.join(data_path, "Cambridge", "cambridge_condo_conversions.csv")

In [3]:
def get_cols_by_suffix(
        data: pd.DataFrame,
        suff: str,
) -> pd.Series:
        return (data
                .columns
                .to_series()
                [lambda x: x.str.endswith(suff)]
                .str
                .replace(suff, ""))

# Boston dataset

In [197]:
boston_df = pd.read_csv(boston_path, index_col=0)
print(f"Boston dimensions: {boston_df.shape}")
boston_df.head()

Boston dimensions: (2245, 114)


,GIS_ID_post,ADDRESS,num_condo_units,PID_prior,CM_ID_prior,ST_NUM_prior,ST_NAME_prior,UNIT_NUM_prior,ZIPCODE_prior,PTYPE_prior,...,SOURCE,PLAN_ID,LAST_EDIT,BND_CHK,NO_MATCH,TOWN_ID,Shape_STArea__,Shape_STLength__,ShapeSTArea,ShapeSTLength
0,100018000.0,239 LEXINGTON ST,5,100018000,NaN,239,LEXINGTON ST,NaN,2128,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100041000,224 PRINCETON ST,5,100041000,NaN,224,PRINCETON ST,NaN,2128.0,NaN,...,ASSESS,NaN,20191107.0,NaN,N,35.0,2337.972168,242.316822,2337.971955,242.316822
2,100046000,212 PRINCETON ST,3,100046000,NaN,212,PRINCETON ST,NaN,2128.0,NaN,...,ASSESS,NaN,20191107.0,NaN,N,35.0,2570.408203,247.412056,2570.408103,247.412056
3,100175000,311 Lexington ST,4,100175000,NaN,311,Lexington ST,NaN,2128.0,NaN,...,ASSESS,NaN,20191107.0,NaN,N,35.0,1908.895508,199.419788,1908.895538,199.419788
4,100177000,5 SHELBY ST,4,100177000,NaN,5,SHELBY ST,NaN,2128.0,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
boston_df.shape

(2245, 114)

## Check cols

In [199]:
prior_cols = get_cols_by_suffix(boston_df, "_prior")
len(prior_cols)

48

In [200]:
post_cols = get_cols_by_suffix(boston_df, "_post")
len(post_cols)

48

In [201]:
# The set of columns is identical after stripping the suffixes.
(prior_cols.sort_values().values == post_cols.sort_values().values).all()

True

In [202]:
common_cols = (boston_df
               .columns
               .to_series()
               .loc[lambda x: ~(x.str.endswith("_prior") | x.str.endswith("_post"))])
common_cols.head()

ADDRESS                    ADDRESS
num_condo_units    num_condo_units
prior_year              prior_year
post_year                post_year
OBJECTID                  OBJECTID
dtype: object

In [203]:
all_bost_cols = pd.concat((
    common_cols.reset_index(drop=True).to_frame(name="col_name").assign(col_type="common"),
    prior_cols.reset_index(drop=True).to_frame(name="col_name").assign(col_type="prior_post"),
))
all_bost_cols

,col_name,col_type
0,ADDRESS,common
1,num_condo_units,common
2,prior_year,common
3,post_year,common
4,OBJECTID,common
...,...,...
43,OVERALL_COND,prior_post
44,HLF_BTH,prior_post
45,BDRM_COND,prior_post
46,HEAT_TYPE,prior_post


In [204]:
(boston_df
 .query("LU_DESC_prior != LU_DESC_post")
 [["ADDRESS", "LU_DESC_prior", "LU_DESC_post"]])

,ADDRESS,LU_DESC_prior,LU_DESC_post
0,239 LEXINGTON ST,THREE-FAM DWELLING,CONDO MAIN
1,224 PRINCETON ST,APT 4-6 UNITS,CONDO MAIN
2,212 PRINCETON ST,TWO-FAM DWELLING,CONDO MAIN
3,311 Lexington ST,THREE-FAM DWELLING,CONDO MAIN
4,5 SHELBY ST,NaN,NaN
...,...,...,...
2240,151 157 FRANKLIN ST,NaN,NaN
2241,15 SPARHAWK ST,NaN,NaN
2242,125 - 127 BROOKS ST,NaN,NaN
2243,340 FANEUIL ST,NaN,NaN


In [205]:
all_cols_wide = pd.concat((
    all_bost_cols.query("col_type == 'common'").reset_index(drop=True).drop(columns="col_type"),
    all_bost_cols.query("col_type == 'prior_post'").reset_index(drop=True).drop(columns="col_type"),
), axis=1)
all_cols_wide

,col_name,col_name
0,ADDRESS,PID
1,num_condo_units,CM_ID
2,prior_year,ST_NUM
3,post_year,ST_NAME
4,OBJECTID,UNIT_NUM
5,LOC_ID,ZIPCODE
6,POLY_TYPE,PTYPE
7,MAP_NO,LU
8,SOURCE,LAND_VALUE
9,PLAN_ID,BLDG_VALUE


## Fix zip codes

What's going on with the `null` zip codes in Tableau? Picked a random address that had a `null` zip code to look for here.

In [206]:
boston_df.query("ADDRESS == '573 BENNINGTON ST'")

,GIS_ID_post,ADDRESS,num_condo_units,PID_prior,CM_ID_prior,ST_NUM_prior,ST_NAME_prior,UNIT_NUM_prior,ZIPCODE_prior,PTYPE_prior,...,SOURCE,PLAN_ID,LAST_EDIT,BND_CHK,NO_MATCH,TOWN_ID,Shape_STArea__,Shape_STLength__,ShapeSTArea,ShapeSTLength
1799,0100879000_,573 BENNINGTON ST,3,0100879000_,NaN,573,BENNINGTON ST,NaN,02128_,104.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
# 450 rows with underscores at the end!
boston_df.loc[lambda x: x.ZIPCODE_prior.str.endswith("_")]

,GIS_ID_post,ADDRESS,num_condo_units,PID_prior,CM_ID_prior,ST_NUM_prior,ST_NAME_prior,UNIT_NUM_prior,ZIPCODE_prior,PTYPE_prior,...,SOURCE,PLAN_ID,LAST_EDIT,BND_CHK,NO_MATCH,TOWN_ID,Shape_STArea__,Shape_STLength__,ShapeSTArea,ShapeSTLength
1795,0100145000_,410 SARATOGA ST,4,0100145000_,NaN,410,SARATOGA ST,NaN,02128_,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1796,0100155000_,269 LEXINGTON ST,3,0100155000_,NaN,269,LEXINGTON ST,NaN,02128_,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1797,0100196000_,276 PRINCETON ST,4,0100196000_,NaN,276,PRINCETON ST,NaN,02128_,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1798,0100437000_,150 WM F MCCLELLAN HW,2,0100437000_,NaN,150,WM F MCCLELLAN HW,NaN,02128_,312.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1799,0100879000_,573 BENNINGTON ST,3,0100879000_,NaN,573,BENNINGTON ST,NaN,02128_,104.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2240,2201469010_,151 157 FRANKLIN ST,2,2201469010_,2201469010_,151 157,FRANKLIN ST,NaN,02134_,995.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2241,2202318000_,15 SPARHAWK ST,2,2202318000_,NaN,15,SPARHAWK ST,NaN,02135_,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2242,2203248020_,125 - 127 BROOKS ST,2,2203248020_,2203248020_,125 - 127,BROOKS ST,NaN,02135_,995.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2243,2203879000_,340 FANEUIL ST,3,2203879000_,NaN,340,FANEUIL ST,NaN,02135_,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Looks like zip codes ending with `_` are causing the problem. Let's also make sure the `_prior` and `_post` zip codes all match.

In [208]:
(boston_df
 .loc[lambda x: x.ZIPCODE_post != x.ZIPCODE_prior]
 [["ADDRESS", "ZIPCODE_post", "ZIPCODE_prior"]])

,ADDRESS,ZIPCODE_post,ZIPCODE_prior
5,278 PRINCETON ST,2128.0,2128
13,82 WORDSWORTH ST,2128.0,2128
17,9 WORDSWORTH ST,2128.0,2128
21,68 HORACE ST,2128.0,2128
25,119 BARNES AV,2128.0,2128
...,...,...,...
1789,35 SNOW ST,2135.0,2135
1791,24 WINSHIP ST,2135.0,2135
1792,18 WINSHIP ST,2135.0,2135
1793,33 WINSHIP ST,2135.0,2135


Interesting that we also see four digit zip codes since they're all supposed to be five digits. Looks like these are Boston zip codes that are missing preceding zeros
(all Boston zip codes start with 0) from the zip codes, which meshes with these getting read in and written out as numerics at some point.
We also see trailing `.0` likely due to storing as a float.

Seems like we have three issues to fix: 
- zip codes ending with `_`
- zip codes ending with `.0`
- 4 digit zip codes missing preceding zeros

In [209]:
# Get some examples for report.
examples = pd.concat((
    boston_df.head(1),
    boston_df.loc[lambda x: x.ZIPCODE_prior.str.endswith("_")].sample(1),
    boston_df.loc[lambda x: x.ZIPCODE_prior.str.endswith(".0")].sample(1),
    boston_df.loc[lambda x: x.ZIPCODE_prior.str.len() == 4].sample(1),
))[["ADDRESS", "ZIPCODE_prior", "ZIPCODE_post"]]
examples

,ADDRESS,ZIPCODE_prior,ZIPCODE_post
0,239 LEXINGTON ST,2128,2128
2193,288 BEECH ST,02131_,02131_
383,20 CHARLESGATE WEST,2215.0,2215.0
1525,2 FOWLE ST,2131,2131.0


In [210]:
def fix_zip(df, col):
    new_zip = (df
               [col]
               .str.replace(".0", "")
               .str.rstrip("_"))

    return pd.Series(np.where(new_zip.str.len() == 4,
                              "0" + new_zip,
                              new_zip))


In [211]:
boston_df["zipcode_prior_fix"] = fix_zip(boston_df, "ZIPCODE_prior")
boston_df.zipcode_prior_fix.value_counts()

zipcode_prior_fix
02127    277
02130    255
02131    233
02128    212
02125    136
02122    126
02124    121
02136    115
02119    111
02129     86
02118     81
02132     75
02135     70
02121     55
02116     54
02120     37
02134     34
02115     31
02215     28
02126     26
02108     21
02114     20
02113     18
02210     10
02110      4
02109      4
02111      3
02199      1
02467      1
Name: count, dtype: int64

In [212]:
# All zip codes are 5 digits now
boston_df.zipcode_prior_fix.str.len().value_counts()

zipcode_prior_fix
5    2245
Name: count, dtype: int64

In [213]:
boston_df["zipcode_post_fix"] = fix_zip(boston_df, "ZIPCODE_post")
boston_df.zipcode_post_fix.value_counts()

zipcode_post_fix
02127    277
02130    255
02131    233
02128    212
02125    136
02122    126
02124    121
02136    115
02119    111
02129     86
02118     81
02132     75
02135     70
02121     55
02116     54
02120     37
02134     34
02115     31
02215     28
02126     26
02108     21
02114     20
02113     18
02210     10
02110      4
02109      4
02111      3
02199      1
02467      1
Name: count, dtype: int64

In [214]:
boston_df.zipcode_post_fix.str.len().value_counts()

zipcode_post_fix
5    2245
Name: count, dtype: int64

In [215]:
(boston_df.zipcode_prior_fix == boston_df.zipcode_post_fix).all()

True

And now the `_prior` and `_post` zip' codes match! So let's just have a single zip code column. Note that we're keeping the malformed `ZIPCODE_prior` column for the purposes of making the illustrative plot.

In [216]:
boston_df = (boston_df
             .rename(columns={"zipcode_prior_fix": "zipcode"})
             .drop(columns=["ZIPCODE_post", "zipcode_post_fix"]))

## Map zip codes to neighborhoods

While zip codes are useful for pinpointing actual locations, most people don't know how these map to the neighborhood names that they're more familiar with thinking in terms of. So let's map the zip codes to neighborhood names.

In [224]:
# Data sourced from here:
# https://owd.boston.gov/wp-content/uploads/2015/07/Neighborhood-Boundaries-and-Zip-Codes.pdf
neighborhood_map = {
    "Allston/Brighton": ["02134", "02135","02163"],
    "Back Bay/Beacon Hill": ["02108","02116", "02117", "02123", "02133", "02199", "02216", "02217", "02295"],
    "Central Boston": ["02101", "02102", "02103", "02104", "02105", "02106", "02107", "02109", "02110", "02111", "02112", "02113", "02114", "02196", "02201", "02202", "02203", "02204", "02205", "02206", "02207", "02208", "02209", "02211", "02212", "02222", "02293"],
    "Charlestown": ["02129"],
    # Had to add this one in manually
    "Chestnut Hill": ["02467"],
    "Dorchester": ["02122", "02124", "02125"],
    "East Boston": ["02128", "02228"],
    "Fenway/Kenmore": ["02115", "02215"],
    "Hyde Park": ["02136"],
    "Jamaica Plain": ["02130"],
    "Mattapan": ["02126"],
    "Roslindale": ["02131"],
    "Roxbury": ["02119", "02120", "02121"],
    "South Boston": ["02127", "02210"],
    "South End": ["02118"],
    "West Roxbury": ["02132"],
}

zip_to_hood = {
    zipcode:hood for hood, zipcodes in neighborhood_map.items() for zipcode in zipcodes
}

In [225]:
boston_df = (boston_df
             .assign(
                 neighborhood=lambda x: x.zipcode.apply(lambda y: zip_to_hood[y])
             ))

In [226]:
boston_df.neighborhood.value_counts()

neighborhood
Dorchester              383
South Boston            287
Jamaica Plain           255
Roslindale              233
East Boston             212
Roxbury                 203
Hyde Park               115
Allston/Brighton        104
Charlestown              86
South End                81
Back Bay/Beacon Hill     76
West Roxbury             75
Fenway/Kenmore           59
Central Boston           49
Mattapan                 26
Chestnut Hill             1
Name: count, dtype: int64

## Check years

Since we're also interested in looking at condo conversion trends over time, let's check the columns related to year of conversion.

In [227]:
boston_df.prior_year.value_counts(dropna=False).sort_index()

prior_year
2015    230
2016    220
2018    297
2019     57
2020     86
2021    253
2022    868
2023    234
Name: count, dtype: int64

In [228]:
boston_df.post_year.value_counts(dropna=False).sort_index()

post_year
2016    230
2017    220
2019    297
2020     57
2021     86
2022    253
2023    868
2024    234
Name: count, dtype: int64

In [229]:
pd.crosstab(boston_df.prior_year, boston_df.post_year)

post_year,2016,2017,2019,2020,2021,2022,2023,2024
prior_year,,,,,,,,
2015,230,0,0,0,0,0,0,0
2016,0,220,0,0,0,0,0,0
2018,0,0,297,0,0,0,0,0
2019,0,0,0,57,0,0,0,0
2020,0,0,0,0,86,0,0,0
2021,0,0,0,0,0,253,0,0
2022,0,0,0,0,0,0,868,0
2023,0,0,0,0,0,0,0,234


Potentially some weirdness going on with year built below, but I don't think that's a critical issue for any of our current analyses. Although if we became interested in something like whether the city was losing its "historical character", then perhaps we'd want to revisit these values.

Weirdness here means year == 0, or one of year built prior or year built post being NA.

In [230]:
boston_df.YR_BUILT_prior.value_counts().sort_index()

YR_BUILT_prior
0.0       57
1800.0     2
1806.0     1
1808.0     1
1824.0     1
          ..
2018.0    16
2019.0    38
2020.0    43
2021.0    32
2022.0     5
Name: count, Length: 139, dtype: int64

In [231]:
(boston_df.YR_BUILT_prior == boston_df.YR_BUILT_post).value_counts()

True     1718
False     527
Name: count, dtype: int64

In [232]:
(boston_df.loc[lambda x: x.YR_BUILT_prior != x.YR_BUILT_post]
 [["ADDRESS", "YR_BUILT_prior", "YR_BUILT_post"]])

,ADDRESS,YR_BUILT_prior,YR_BUILT_post
1,224 PRINCETON ST,1900.0,2021.0
9,398 BREMEN ST,1925.0,2020.0
13,82 WORDSWORTH ST,2016.0,NaN
14,76 WORDSWORTH ST,NaN,2021.0
16,58 BYRON ST,NaN,2019.0
...,...,...,...
2226,853 BEACON ST,1899.0,0.0
2232,1501 COMMONWEALTH AV,2014.0,0.0
2233,261 265 WASHINGTON ST,1964.0,1968.0
2239,165 EVERETT ST,0.0,2014.0


## Building type

According to README on github, `LU_DESC` is a human-readable description of the land use code in the `LU` column, which sounds relevant.

In [234]:
boston_df.LU_prior.value_counts(dropna=False)

LU_prior
R3         565
R2         463
CD         405
R1         155
CM         130
R4         118
E          107
A           92
C           61
RL          52
RC          52
CC          19
CL          10
EA           7
I            5
RL - RL      4
Name: count, dtype: int64

In [235]:
boston_df.LU_post.value_counts(dropna=False)

LU_post
CM         1610
E           137
CD          127
R1          117
R3           86
C            32
R2           32
RL - RL      24
A            23
RC           11
CP           10
R4            8
RL            8
CL            7
EA            7
CC            4
I             2
Name: count, dtype: int64

In [236]:
boston_df.LU_prior.isin(boston_df.LU_post).all()

True

In [239]:
boston_df.LU_DESC_prior.value_counts(dropna=False)

LU_DESC_prior
NaN                        890
RESIDENTIAL CONDO          388
THREE-FAM DWELLING         289
TWO-FAM DWELLING           229
SINGLE FAM DWELLING        128
APT 4-6 UNITS               70
CONDO MAIN                  51
RES /COMMERCIAL USE         25
LUXURY APARTMENT            23
APT 7-30 UNITS              22
RES LAND (Unusable)         18
OTHER EXEMPT BLDG           11
RETAIL CONDO                10
RESIDENTIAL LAND             7
OFFICE CONDO                 7
RET/WHSL/SERVICE             6
CHARITABLE Organization      5
OLD WHSE, GARAGE             5
SUBSD HOUSING S- 8           5
BOS HOUSING AUTHOR           4
CHURCH, SYNAGOGUE            4
COMMERCIAL LAND              3
COMM MULTI-USE               3
OFFICE 3-9 STORY             2
CELL CARRIER                 2
Commonwealth of Mass         2
PARKING LOT                  2
COLLEGE (academic)           2
REPAIR GARAGE                2
RESTAURANT/Cafeteria         2
COMM  CONDO                  2
WAREHOUSE /DISTRIB       

In [240]:
# Lots of NAs, are they at least consistent? Need another variable
# since NaN == NaN is False.
boston_df = (boston_df
             .assign(
                 land_use_prior=lambda x: np.where(x.LU_DESC_prior.isna(),
                                                   "none",
                                                   x.LU_DESC_prior),
                 land_use_post=lambda x: np.where(x.LU_DESC_post.isna(),
                                                  "none",
                                                  x.LU_DESC_post),
             ))
(boston_df.land_use_prior == boston_df.land_use_post).value_counts()

False    1131
True     1114
Name: count, dtype: int64

In [241]:
# What are the differences? Seems like some are what we expect (type X -> condo)
# others, perhaps not so much, e.g. COLLEGE (academic) ->	PRIV SCHOOL /COLLEGE
(boston_df
 .query("land_use_prior != land_use_post")
 [["ADDRESS", "land_use_prior", "land_use_post"]])

,ADDRESS,land_use_prior,land_use_post
0,239 LEXINGTON ST,THREE-FAM DWELLING,CONDO MAIN
1,224 PRINCETON ST,APT 4-6 UNITS,CONDO MAIN
2,212 PRINCETON ST,TWO-FAM DWELLING,CONDO MAIN
3,311 Lexington ST,THREE-FAM DWELLING,CONDO MAIN
8,6 SHELBY ST,THREE-FAM DWELLING,CONDO MAIN
...,...,...,...
1785,19 21 ANSELM TE,none,CONDO MAIN
1787,2121 COMMONWEALTH AV,COLLEGE (academic),PRIV SCHOOL /COLLEGE
1788,29 NANTASKET AV,none,MULTIPLE BLDGS/1 LOT
1790,38 SHEPARD ST,THREE-FAM DWELLING,CONDO MAIN


In [242]:
(boston_df.LU_DESC_prior == boston_df.LU_DESC_post).value_counts()

False    1935
True      310
Name: count, dtype: int64

In [246]:
boston_df.LU_prior.isna().any()

False

In [247]:
boston_df.LU_post.isna().any()

False

In [248]:
land_use_map = {
    'A': 'Residential 7 or more units',
    'AH': 'Agricultural/Horticultural',
    'C': 'Commercial',
    'CC': 'Commercial condominium',
    'CD': 'Residential condominium unit',
    'CL': 'Commercial land',
    'CM': 'Condominium main (physical strucure housing all related condo units  with no assessed value)',
    'CP': 'Condo parking',
    'E': 'Tax-exempt',
    'EA': 'Tax-exempt (121A)',
    'I': 'Industrial',
    'R1': 'Residential 1-family',
    'R2': 'Residential 2-family',
    'R3': 'Residential 3-family',
    'R4': 'Residential 4 or more family',
    'RC': 'Mixed use (res. and comm.)',
    'RL': 'Residential land',
    'RL - RL': 'Residential land',
}

land_use_map_condensed = {
    # Note: AH doesn't actually show up in our dataset
    'AH': 'Agricultural',
    'I': 'Industrial',

    'C': 'Commercial',
    'CC': 'Commercial',
    'CL': 'Commercial',

    'CM': 'Condominium',
    'CD': 'Condominium',
    'CP': 'Condominium',

    'E': 'Tax-exempt',
    'EA': 'Tax-exempt',

    'A':  'Residential',
    'R1': 'Residential',
    'R2': 'Residential',
    'R3': 'Residential',
    'R4': 'Residential',
    'RC': 'Residential',
    'RL': 'Residential',
    'RL - RL': 'Residential',
 }

In [249]:
boston_df = (boston_df
            .assign(
                land_use_prior=lambda x: x.LU_prior.apply(lambda y: land_use_map[y]),
                land_use_post=lambda x: x.LU_post.apply(lambda y: land_use_map[y]),
                land_use_condensed_prior=lambda x: x.LU_prior.apply(lambda y: land_use_map_condensed[y]),
                land_use_condensed_post=lambda x: x.LU_post.apply(lambda y: land_use_map_condensed[y]),
            ))

(boston_df
 [["ADDRESS", "LU_prior", "land_use_prior", "land_use_condensed_prior"]]
 .sample(n=10, random_state=42))

,ADDRESS,LU_prior,land_use_prior,land_use_condensed_prior
1560,4155 WASHINGTON ST,CC,Commercial condominium,Commercial
482,515 E FIRST ST,RC,Mixed use (res. and comm.),Residential
1414,55 MILTON ST,R1,Residential 1-family,Residential
247,47 N MARGIN ST,A,Residential 7 or more units,Residential
2030,74 HIGHLAND ST,A,Residential 7 or more units,Residential
2143,1436 - 144 HYDE PARK AV,CM,Condominium main (physical strucure housing al...,Condominium
1675,61 DENT ST,R2,Residential 2-family,Residential
700,31 IROQUOIS ST,CD,Residential condominium unit,Condominium
1210,38 ARMANDINE ST,R3,Residential 3-family,Residential
1557,239 WACHUSETT ST,CD,Residential condominium unit,Condominium


In [250]:
stayed_condo = (boston_df
                .query("land_use_condensed_prior == 'Condominium' and land_use_condensed_post == 'Condominium'"))
len(stayed_condo)

533

In [251]:
pd.crosstab(stayed_condo.LU_prior, stayed_condo.LU_post)

LU_post,CD,CM
LU_prior,,
CD,61,342
CM,13,117


In [252]:
(boston_df
 .loc[lambda x: x.LU_prior == "CD"]
 [["ADDRESS", "LU_prior", "LU_post", "LU_DESC_prior", "LU_DESC_post"]])

,ADDRESS,LU_prior,LU_post,LU_DESC_prior,LU_DESC_post
13,82 WORDSWORTH ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
31,40 WHITBY ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
48,40 EUTAW ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
69,181 FALCON ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
73,123 FALCON ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
...,...,...,...,...,...
1881,301 -319 COLUMBUS AV,CD,CD,NaN,NaN
1884,230 -232 W NEWTON ST,CD,CD,NaN,NaN
2015,238 S HUNTINGTON AV,CD,CD,NaN,NaN
2187,95 WACHUSETT ST,CD,CD,NaN,NaN


In [253]:
(boston_df
 .loc[lambda x: x.LU_prior == "CD"]
 .LU_post.value_counts())

LU_post
CM    342
CD     61
E       1
R1      1
Name: count, dtype: int64

In [254]:
(boston_df
 .loc[lambda x: (x.LU_prior == "CD") & (x.LU_post != "CD")]
 [["ADDRESS", "LU_prior", "LU_post", "LU_DESC_prior", "LU_DESC_post"]])

,ADDRESS,LU_prior,LU_post,LU_DESC_prior,LU_DESC_post
13,82 WORDSWORTH ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
31,40 WHITBY ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
48,40 EUTAW ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
69,181 FALCON ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
73,123 FALCON ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
...,...,...,...,...,...
1779,104 FOSTER TE,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
1780,1997 COMMONWEALTH AV,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
1782,22 SHANLEY ST,CD,CM,RESIDENTIAL CONDO,CONDO MAIN
1784,25 ROGERS PARK AV,CD,CM,RESIDENTIAL CONDO,CONDO MAIN


In [255]:
(boston_df
 .loc[lambda x: x.LU_prior.str.contains("RL")]
 [["ADDRESS", "LU_prior", "LU_post", "LU_DESC_prior", "LU_DESC_post", "prior_year"]]
 .head(n=10))


,ADDRESS,LU_prior,LU_post,LU_DESC_prior,LU_DESC_post,prior_year
16,58 BYRON ST,RL,CM,RESIDENTIAL LAND,CONDO MAIN,2021
22,689 BENNINGTON ST,RL,CM,RESIDENTIAL LAND,CONDO MAIN,2021
198,36 BELMONT ST,RL,CM,RESIDENTIAL LAND,CONDO MAIN,2022
219,74 TREMONT ST,RL,R1,NaN,NaN,2018
388,51 SILVER ST,RL,CM,RESIDENTIAL LAND,CONDO MAIN,2021
425,341 E ST,RL,CM,NaN,NaN,2018
479,175 K ST,RL,CM,NaN,NaN,2018
488,650 E THIRD ST,RL,R2,RES LAND (Unusable),TWO-FAM DWELLING,2022
555,3 TELEGRAPH ST,RL,CD,RESIDENTIAL LAND,RESIDENTIAL CONDO,2022
595,342 K ST,RL,RL - RL,RES ANCILL IMPROVEMT,RES LAND (Unusable),2022


In [ ]:
boston_df.num_condo_units

In [262]:
(boston_df
 .query("land_use_condensed_prior == 'Residential' and land_use_condensed_post == 'Condominium'")
 .num_condo_units
 .value_counts()
 .sort_index())

num_condo_units
2       56
3      418
4      449
5       84
6       25
7       38
8       21
9       10
10      22
11       3
12       4
13       5
14       5
17       1
18       3
19       7
20       3
25       1
27       2
32       1
35       1
37       1
39       1
41       2
45       1
48       1
50       1
55       1
56       1
61       1
75       1
82       1
83       1
85       1
87       1
110      1
111      1
139      1
169      1
280      1
Name: count, dtype: int64

In [263]:
(boston_df
 .query("land_use_condensed_prior == 'Residential' and land_use_condensed_post == 'Condominium' and neighborhood == 'South Boston'")
 .num_condo_units
 .value_counts()
 .sort_index())

num_condo_units
2      10
3      54
4      47
5      14
6       6
7       5
8       5
9       1
10      5
11      2
12      2
13      1
14      2
17      1
18      1
19      2
20      2
25      1
27      2
32      1
48      1
50      1
55      1
75      1
85      1
110     1
280     1
Name: count, dtype: int64

In [267]:
boston_df.query("num_condo_units == 280")

,GIS_ID_post,ADDRESS,num_condo_units,PID_prior,CM_ID_prior,ST_NUM_prior,ST_NAME_prior,UNIT_NUM_prior,ZIPCODE_prior,PTYPE_prior,...,Shape_STArea__,Shape_STLength__,ShapeSTArea,ShapeSTLength,zipcode,neighborhood,land_use_prior,land_use_post,land_use_condensed_prior,land_use_condensed_post
1946,0602670010_,22 LIBERTY DR,280,0602671019_,NaN,22,LIBERTY DR,NaN,02210_,13.0,...,NaN,NaN,NaN,NaN,02210,South Boston,Mixed use (res. and comm.),Condominium main (physical strucure housing al...,Residential,Condominium


In [256]:
with open(os.path.join(data_path, "Boston", "boston_condo_conversions.fixed.csv"), "w") as fh:
    boston_df.to_csv(fh, index=False)

In [257]:
boston_df.ZIPCODE_prior

0         2128
1       2128.0
2       2128.0
3       2128.0
4       2128.0
         ...  
2240    02134_
2241    02135_
2242    02135_
2243    02135_
2244    02135_
Name: ZIPCODE_prior, Length: 2245, dtype: object

# Cambridge dataset

In [28]:
camb_df = pd.read_csv(cambridge_path, index_col=0)
camb_df.head()

,Address,num_condo_units,PID_prior,GISID_prior,BldgNum_prior,Unit_prior,StateClassCode_prior,PropertyClass_prior,Zoning_prior,Map/Lot_prior,...,ML,SOURCE,created_user,Editor,LOT,created_date,PLAN_ID,last_edited_date,last_edited_user,geometry
0,10 ELLERY ST,4,8053,116-119,1,NaN,104,TWO-FAM-RES,"=""C-1""","=""116-119""",...,116-119,SUBDIV,NaN,RMF,119.0,NaN,1019/2013,2016-10-12,SSWEENEY,POLYGON ((-71.11093353988682 42.37015235857614...
1,12 AVON PL,3,17759,211-15,1,NaN,104,TWO-FAM-RES,"=""A-2""","=""211-15""",...,211-15,ASSESS,NaN,NaN,15.0,NaN,NaN,2023-09-26,SSWEENEY,POLYGON ((-71.12342903741833 42.38470535494655...
2,12 RINDGEFIELD ST,3,16472,196-91,1,NaN,104,TWO-FAM-RES,"=""B""","=""196-91""",...,196-91,ASSESS,NaN,NaN,91.0,NaN,NaN,2023-09-26,SSWEENEY,POLYGON ((-71.12739716910473 42.39358022094926...
3,133-135 FRESH POND PKWY,3,21305,259-17,1,NaN,104,TWO-FAM-RES,"=""B""","=""259-17""",...,259-17,ASSESS,NaN,NaN,17.0,NaN,NaN,2023-09-26,SSWEENEY,POLYGON ((-71.14419522238404 42.38096471923172...
4,139 CHARLES ST,3,690,17-20,1,NaN,101,SNGL-FAM-RES,"=""C-1""","=""17-20""",...,17-20,ASSESS,NaN,NaN,20.0,NaN,NaN,2023-09-26,SSWEENEY,POLYGON ((-71.08202646412141 42.36802796241646...


In [29]:
camb_df.shape

(285, 161)

## Col explore

In [30]:
camb_prior_cols = get_cols_by_suffix(camb_df, "_prior")
len(camb_prior_cols)

63

In [31]:
camb_post_cols = get_cols_by_suffix(camb_df, "_post")
len(camb_post_cols)

63

In [32]:
# Again, set of columns is identical after stripping suffixes.
(camb_prior_cols.sort_values().values == camb_post_cols.sort_values().values).all()

True

In [39]:
common_cols = (camb_df
               .columns
               .to_series()
               .loc[lambda x: ~(x.str.endswith("_prior") | x.str.endswith("_post"))])
common_cols

Address                          Address
num_condo_units          num_condo_units
Owners                            Owners
Owner_Name                    Owner_Name
Owner_CoOwnerName      Owner_CoOwnerName
Owner_Address              Owner_Address
Owner_Address2            Owner_Address2
Owner_City                    Owner_City
Owner_State                  Owner_State
Owner_Zip                      Owner_Zip
prior_year                    prior_year
post_year                      post_year
PropertyTaxAmount      PropertyTaxAmount
Map/Lot                          Map/Lot
Book/Page                      Book/Page
Exterior_occupancy    Exterior_occupancy
MapLot                            MapLot
BookPage                        BookPage
Exterior_Occupancy    Exterior_Occupancy
POLY_TYPE                      POLY_TYPE
MAP                                  MAP
UYEAR                              UYEAR
LOC_ID                            LOC_ID
GlobalID                        GlobalID
EditDate        

In [34]:
all_camb_cols = pd.concat((
    common_cols.reset_index(drop=True).to_frame(name="col_name").assign(col_type="common"),
    camb_prior_cols.reset_index(drop=True).to_frame(name="col_name").assign(col_type="prior_post"),
))
all_camb_cols

,col_name,col_type
0,Address,common
1,num_condo_units,common
2,Owners,common
3,Owner_Name,common
4,Owner_CoOwnerName,common
...,...,...
58,Owner_Zip,prior_post
59,PropertyTaxAmount,prior_post
60,MapLot,prior_post
61,BookPage,prior_post


In [41]:
pd.pivot(all_camb_cols, columns="col_type", values="col_name").reset_index(drop=True)

col_type,common,prior_post
0,Address,PID
1,num_condo_units,GISID
2,Owners,BldgNum
3,Owner_Name,Unit
4,Owner_CoOwnerName,StateClassCode
5,Owner_Address,PropertyClass
6,Owner_Address2,Zoning
7,Owner_City,Map/Lot
8,Owner_State,LandArea
9,Owner_Zip,YearOfAssessment


In [46]:
camb_df.columns.to_list()

['Address',
 'num_condo_units',
 'PID_prior',
 'GISID_prior',
 'BldgNum_prior',
 'Unit_prior',
 'StateClassCode_prior',
 'PropertyClass_prior',
 'Zoning_prior',
 'Map/Lot_prior',
 'LandArea_prior',
 'YearOfAssessment_prior',
 'TaxDistrict_prior',
 'ResidentialExemption_prior',
 'BuildingValue_prior',
 'LandValue_prior',
 'AssessedValue_prior',
 'SalePrice_prior',
 'Book/Page_prior',
 'SaleDate_prior',
 'PreviousAssessedValue_prior',
 'Owners',
 'Exterior_Style_prior',
 'Exterior_occupancy_prior',
 'Exterior_NumStories_prior',
 'Exterior_WallType_prior',
 'Exterior_WallHeight_prior',
 'Exterior_RoofType_prior',
 'Exterior_RoofMaterial_prior',
 'Exterior_FloorLocation_prior',
 'Exterior_View_prior',
 'Interior_LivingArea_prior',
 'Interior_NumUnits_prior',
 'Interior_TotalRooms_prior',
 'Interior_Bedrooms_prior',
 'Interior_Kitchens_prior',
 'Interior_FullBaths_prior',
 'Interior_HalfBaths_prior',
 'Interior_Fireplaces_prior',
 'Interior_Flooring_prior',
 'Interior_Layout_prior',
 'Inter